# Notebook 4: Real-World Dashboard Example

Welcome to the final notebook! Let's put everything together and build a complete, real-world dashboard.

In this notebook, you'll learn:
- Building a production-ready dashboard
- Working with real datasets
- Advanced callbacks and interactivity
- Performance optimization
- Best practices for deployment

## Setup and Imports

In [ ]:
import dash
from dash import html, dcc, Input, Output, State, callback_context
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

try:
    import dash_bootstrap_components as dbc
    print("dash-bootstrap-components available!")
    DBC_AVAILABLE = True
except ImportError:
    print("dash-bootstrap-components not available. Some features will be limited.")
    DBC_AVAILABLE = False

print("All imports successful!")

## Creating a Realistic Dataset

Let's create a comprehensive dataset that simulates an e-commerce business.

In [ ]:
# Generate realistic e-commerce data
np.random.seed(42)

# Date range: last 12 months
end_date = datetime.now()
start_date = end_date - timedelta(days=365)
dates = pd.date_range(start=start_date, end=end_date, freq='D')

# Product categories
categories = ['Electronics', 'Clothing', 'Home & Garden', 'Books', 'Sports', 'Toys']
regions = ['North', 'South', 'East', 'West']

# Generate sales data
data = []
for date in dates:
    for category in categories:
        for region in regions:
            # Add some seasonality and trends
            base_sales = np.random.randint(1000, 5000)
            trend = (date - start_date).days * 2  # Growth trend
            seasonality = 1000 * np.sin(2 * np.pi * (date - start_date).days / 365)  # Yearly pattern
            
            sales = int(base_sales + trend + seasonality + np.random.normal(0, 500))
            sales = max(sales, 0)  # Ensure non-negative
            
            units = int(sales / np.random.randint(20, 100))
            
            data.append({
                'Date': date,
                'Category': category,
                'Region': region,
                'Sales': sales,
                'Units': units,
                'Returns': int(units * np.random.uniform(0.02, 0.08))  # 2-8% return rate
            })

df_sales = pd.DataFrame(data)
df_sales['Month'] = df_sales['Date'].dt.to_period('M').astype(str)
df_sales['Profit'] = (df_sales['Sales'] * np.random.uniform(0.15, 0.35, len(df_sales))).astype(int)

print(f"Generated {len(df_sales):,} records")
print(f"Date range: {df_sales['Date'].min()} to {df_sales['Date'].max()}")
print(f"\nSample data:")
print(df_sales.head())

## Save Sample Data

Let's save this data so it can be used independently.

In [ ]:
# Save to CSV
df_sales.to_csv('../data/ecommerce_sales.csv', index=False)
print("Data saved to ../data/ecommerce_sales.csv")

## Building the Complete Dashboard

Now let's build a comprehensive e-commerce analytics dashboard.

In [ ]:
if DBC_AVAILABLE:
    # Full dashboard with Bootstrap
    app = dash.Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])
    
    # Color scheme
    colors = {
        'primary': '#2c3e50',
        'success': '#27ae60',
        'danger': '#e74c3c',
        'info': '#3498db',
        'warning': '#f39c12'
    }
    
    # Helper function to create KPI card
    def create_kpi_card(title, value, icon, color, change=None):
        card_content = [
            html.H6(title, className="text-muted mb-2"),
            html.H3(value, className="mb-0", style={'color': color})
        ]
        
        if change:
            change_color = 'success' if change >= 0 else 'danger'
            card_content.append(
                html.Small(f"{'↑' if change >= 0 else '↓'} {abs(change):.1f}% from last month",
                          className=f"text-{change_color}")
            )
        
        return dbc.Card([
            dbc.CardBody(card_content)
        ], className="shadow-sm")
    
    # Navigation bar
    navbar = dbc.NavbarSimple(
        brand="E-Commerce Analytics Dashboard",
        brand_href="#",
        color="primary",
        dark=True,
        className="mb-4"
    )
    
    # Filters section
    filters = dbc.Card([
        dbc.CardBody([
            dbc.Row([
                dbc.Col([
                    html.Label("Date Range:", className="fw-bold"),
                    dcc.DatePickerRange(
                        id='date-range',
                        start_date=df_sales['Date'].max() - timedelta(days=90),
                        end_date=df_sales['Date'].max(),
                        display_format='YYYY-MM-DD'
                    )
                ], md=4),
                
                dbc.Col([
                    html.Label("Category:", className="fw-bold"),
                    dcc.Dropdown(
                        id='category-filter',
                        options=[{'label': 'All Categories', 'value': 'All'}] + 
                                [{'label': cat, 'value': cat} for cat in categories],
                        value='All',
                        clearable=False
                    )
                ], md=4),
                
                dbc.Col([
                    html.Label("Region:", className="fw-bold"),
                    dcc.Dropdown(
                        id='region-filter',
                        options=[{'label': 'All Regions', 'value': 'All'}] + 
                                [{'label': reg, 'value': reg} for reg in regions],
                        value='All',
                        clearable=False
                    )
                ], md=4)
            ])
        ])
    ], className="mb-4")
    
    # App layout
    app.layout = dbc.Container([
        navbar,
        filters,
        
        # KPI Cards
        dbc.Row([
            dbc.Col(html.Div(id='kpi-sales'), md=3),
            dbc.Col(html.Div(id='kpi-profit'), md=3),
            dbc.Col(html.Div(id='kpi-orders'), md=3),
            dbc.Col(html.Div(id='kpi-returns'), md=3)
        ], className="mb-4"),
        
        # Charts Row 1
        dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardHeader(html.H5("Sales Trend", className="mb-0")),
                    dbc.CardBody([dcc.Graph(id='sales-trend')])
                ], className="shadow-sm")
            ], md=8),
            
            dbc.Col([
                dbc.Card([
                    dbc.CardHeader(html.H5("Sales by Category", className="mb-0")),
                    dbc.CardBody([dcc.Graph(id='category-pie')])
                ], className="shadow-sm")
            ], md=4)
        ], className="mb-4"),
        
        # Charts Row 2
        dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardHeader(html.H5("Regional Performance", className="mb-0")),
                    dbc.CardBody([dcc.Graph(id='regional-bar')])
                ], className="shadow-sm")
            ], md=6),
            
            dbc.Col([
                dbc.Card([
                    dbc.CardHeader(html.H5("Top Products", className="mb-0")),
                    dbc.CardBody([dcc.Graph(id='top-products')])
                ], className="shadow-sm")
            ], md=6)
        ])
    ], fluid=True, className="px-4")
    
    # Main callback
    @app.callback(
        [Output('kpi-sales', 'children'),
         Output('kpi-profit', 'children'),
         Output('kpi-orders', 'children'),
         Output('kpi-returns', 'children'),
         Output('sales-trend', 'figure'),
         Output('category-pie', 'figure'),
         Output('regional-bar', 'figure'),
         Output('top-products', 'figure')],
        [Input('date-range', 'start_date'),
         Input('date-range', 'end_date'),
         Input('category-filter', 'value'),
         Input('region-filter', 'value')]
    )
    def update_dashboard(start_date, end_date, category, region):
        # Filter data
        filtered_df = df_sales.copy()
        filtered_df = filtered_df[
            (filtered_df['Date'] >= start_date) & 
            (filtered_df['Date'] <= end_date)
        ]
        
        if category != 'All':
            filtered_df = filtered_df[filtered_df['Category'] == category]
        
        if region != 'All':
            filtered_df = filtered_df[filtered_df['Region'] == region]
        
        # Calculate KPIs
        total_sales = filtered_df['Sales'].sum()
        total_profit = filtered_df['Profit'].sum()
        total_orders = filtered_df['Units'].sum()
        total_returns = filtered_df['Returns'].sum()
        
        # Calculate changes (comparing to previous period)
        period_days = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days
        prev_start = pd.to_datetime(start_date) - timedelta(days=period_days)
        prev_end = pd.to_datetime(start_date)
        
        prev_df = df_sales[
            (df_sales['Date'] >= prev_start) & 
            (df_sales['Date'] < prev_end)
        ]
        
        if category != 'All':
            prev_df = prev_df[prev_df['Category'] == category]
        if region != 'All':
            prev_df = prev_df[prev_df['Region'] == region]
        
        prev_sales = prev_df['Sales'].sum()
        sales_change = ((total_sales - prev_sales) / prev_sales * 100) if prev_sales > 0 else 0
        
        # Create KPI cards
        kpi1 = create_kpi_card("Total Sales", f"${total_sales:,.0f}", "💰", colors['success'], sales_change)
        kpi2 = create_kpi_card("Total Profit", f"${total_profit:,.0f}", "📈", colors['info'])
        kpi3 = create_kpi_card("Orders", f"{total_orders:,}", "🛒", colors['primary'])
        kpi4 = create_kpi_card("Returns", f"{total_returns:,}", "↩️", colors['warning'])
        
        # Sales trend chart
        daily_sales = filtered_df.groupby('Date')['Sales'].sum().reset_index()
        fig_trend = px.line(daily_sales, x='Date', y='Sales',
                           title='',
                           template='plotly_white')
        fig_trend.update_traces(line_color=colors['primary'], line_width=2)
        fig_trend.update_layout(hovermode='x unified', margin=dict(t=10, b=10))
        
        # Category pie chart
        category_sales = filtered_df.groupby('Category')['Sales'].sum().reset_index()
        fig_pie = px.pie(category_sales, values='Sales', names='Category',
                        title='',
                        hole=0.4,
                        template='plotly_white')
        fig_pie.update_layout(margin=dict(t=10, b=10))
        
        # Regional bar chart
        regional_data = filtered_df.groupby('Region').agg({
            'Sales': 'sum',
            'Profit': 'sum'
        }).reset_index()
        
        fig_regional = go.Figure()
        fig_regional.add_trace(go.Bar(name='Sales', x=regional_data['Region'], 
                                      y=regional_data['Sales'], marker_color=colors['primary']))
        fig_regional.add_trace(go.Bar(name='Profit', x=regional_data['Region'], 
                                      y=regional_data['Profit'], marker_color=colors['success']))
        fig_regional.update_layout(barmode='group', template='plotly_white',
                                  margin=dict(t=10, b=10))
        
        # Top products (categories)
        top_products = filtered_df.groupby('Category')['Sales'].sum().sort_values(ascending=True).tail(6)
        fig_top = px.bar(x=top_products.values, y=top_products.index,
                        orientation='h',
                        title='',
                        template='plotly_white',
                        color=top_products.values,
                        color_continuous_scale='Blues')
        fig_top.update_layout(showlegend=False, margin=dict(t=10, b=10))
        fig_top.update_xaxes(title='Sales ($)')
        fig_top.update_yaxes(title='')
        
        return kpi1, kpi2, kpi3, kpi4, fig_trend, fig_pie, fig_regional, fig_top
    
    print("Complete dashboard created!")
    print("Run with: app.run_server(debug=True)")
    
else:
    print("Install dash-bootstrap-components for the full dashboard experience")
    print("pip install dash-bootstrap-components")

## Performance Optimization Tips

For production dashboards, consider these optimization techniques:

### 1. Data Caching

In [ ]:
from functools import lru_cache

# Cache expensive data operations
@lru_cache(maxsize=32)
def load_and_process_data(start_date, end_date):
    """Load and filter data with caching"""
    df = pd.read_csv('../data/ecommerce_sales.csv', parse_dates=['Date'])
    filtered = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    return filtered

print("Caching function defined")

### 2. Using dcc.Store for Client-Side Data

In [ ]:
# Example layout with dcc.Store
optimized_layout = html.Div([
    # Store data in browser
    dcc.Store(id='stored-data'),
    
    # Filters
    dcc.Dropdown(id='filter-dropdown'),
    
    # Charts that use stored data
    dcc.Graph(id='chart-1'),
    dcc.Graph(id='chart-2')
])

print("Optimized layout with data storage defined")

### 3. Loading States

Show loading indicators for better UX:

In [ ]:
# Add loading component
loading_layout = html.Div([
    dcc.Loading(
        id="loading",
        type="default",  # or "circle", "dot", "cube"
        children=dcc.Graph(id='my-graph')
    )
])

print("Loading indicator example defined")

## Deployment Preparation

### Creating a Standalone App File

For deployment, save your app as a separate Python file:

In [ ]:
# This is what your app.py should look like
app_code = '''
import dash
from dash import html, dcc, Input, Output
import plotly.express as px
import pandas as pd

# Load data
df = pd.read_csv('data/ecommerce_sales.csv', parse_dates=['Date'])

# Initialize app
app = dash.Dash(__name__)
server = app.server  # For deployment

# Define layout
app.layout = html.Div([
    html.H1("My Dashboard"),
    dcc.Graph(id='my-graph')
])

# Callbacks
@app.callback(
    Output('my-graph', 'figure'),
    Input('my-graph', 'id')
)
def update_graph(graph_id):
    fig = px.line(df, x='Date', y='Sales')
    return fig

# Run server
if __name__ == '__main__':
    app.run_server(debug=True)
'''

print("Example app structure shown above")

## Exercise 4: Build Your Complete Dashboard

Create a production-ready dashboard with:
1. Real data (use the e-commerce dataset or your own)
2. Multiple interactive filters
3. At least 4-5 different visualizations
4. KPI cards showing key metrics
5. Professional styling
6. Loading indicators
7. Responsive layout

In [ ]:
# Your code here
# TODO: Build your complete dashboard



## Deployment Options

### 1. Render (Free)
```bash
# Add Procfile
web: gunicorn app:server

# Update requirements.txt
gunicorn
```

### 2. Heroku
```bash
# Create Procfile
web: gunicorn app:server

# Deploy
heroku create my-dash-app
git push heroku main
```

### 3. Docker
```dockerfile
FROM python:3.9-slim
WORKDIR /app
COPY requirements.txt .
RUN pip install -r requirements.txt
COPY . .
CMD ["gunicorn", "app:server", "-b", "0.0.0.0:8050"]
```

## Best Practices Summary

### Code Organization
- Separate layout, callbacks, and data processing
- Use helper functions for repeated code
- Keep callbacks focused and simple

### Performance
- Cache expensive operations
- Use dcc.Store for shared data
- Minimize callback chains
- Aggregate data before visualizing

### User Experience
- Add loading indicators
- Provide clear labels and instructions
- Use responsive design
- Handle errors gracefully

### Security
- Validate user inputs
- Don't expose sensitive data
- Use environment variables for secrets
- Set appropriate CORS policies

### Maintenance
- Document your code
- Use version control
- Test your callbacks
- Monitor performance in production

## Key Takeaways

Congratulations! You've completed the Plotly Dash workshop. You now know:

✅ How to build production-ready dashboards  
✅ Working with real datasets  
✅ Advanced callbacks and interactions  
✅ Performance optimization techniques  
✅ Best practices for deployment  
✅ Security and maintenance considerations  

## What's Next?

1. **Build Your Own Project**
   - Choose a dataset you're interested in
   - Plan your dashboard layout
   - Implement features incrementally
   - Deploy and share!

2. **Explore Advanced Topics**
   - Real-time data updates
   - Authentication and authorization
   - Database integration
   - Advanced animations

3. **Join the Community**
   - Share your dashboards
   - Help others learn
   - Contribute to Dash projects

## Thank You!

Thank you for participating in this workshop. We hope you found it valuable and feel confident building your own dashboards.

**Keep learning and building! 🚀📊**

## Additional Resources

- [Plotly Dash Documentation](https://dash.plotly.com/)
- [Dash Community Forum](https://community.plotly.com/)
- [Dash Sample Apps](https://github.com/plotly/dash-sample-apps)
- [Plotly Python Graphing Library](https://plotly.com/python/)
- [Dash Enterprise](https://plotly.com/dash/) (for business needs)